In [1]:
# import
import os
import sys
from datetime import timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import plotly
import plotly.graph_objs as go

# import src.mongos as mg
from src.kobis import DB
from src.kobis import (get_boxoffice, is_holiday)

plotly.offline.init_notebook_mode(connected=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

# Const

In [2]:
# Const
docs_daily = 'daily_boxoffice'
movie_names = [
    '어벤져스',
    '어벤져스: 에이지 오브 울트론',
    '어벤져스 인피니티 워',
    '어벤져스: 엔드게임',
]
nm_avengers1 = '어벤져스'
nm_avengers2 = '어벤져스: 에이지 오브 울트론'
nm_avengers3 = '어벤져스: 인피니티 워'
nm_avengers4 = '어벤져스: 엔드게임'
nm_myungryang = '명량'
nm_with_god = '신과함께-죄와 벌'
nm_extreme_job = '극한직업'
nm_ = ''

# Load

In [3]:
# box
raw = get_boxoffice(trim=False)
raw.head()

,audiAcc,audiChange,audiCnt,audiInten,movieCd,movieNm,openDt,rank,rankInten,rankOldAndNew,rnum,salesAcc,salesAmt,salesChange,salesInten,salesShare,scrnCnt,showCnt,targetDt,openDt_f,targetDt_f,isHoliday
0,870,100,870,870,20030191,매트릭스3 레볼루션,2003-11-05,1,0,NEW,1,7676000,7676000,100,7676000,62.6,6,16,20031111,2003-11-05,2003-11-11,False
1,177,100,177,177,20030152,위대한 유산,2003-10-24,2,0,NEW,2,1658000,1658000,100,1658000,13.5,1,3,20031111,2003-10-24,2003-11-11,False
2,116,100,116,116,20030154,황산벌,2003-10-17,3,0,NEW,3,1257000,1257000,100,1257000,10.3,1,3,20031111,2003-10-17,2003-11-11,False
3,102,100,102,102,20030247,아이덴티티,2003-10-31,4,0,NEW,4,821000,821000,100,821000,6.7,1,4,20031111,2003-10-31,2003-11-11,False
4,60,100,60,60,20030127,스캔들-조선남녀상열지사,2003-10-02,5,0,NEW,5,571000,571000,100,571000,4.7,1,3,20031111,2003-10-02,2003-11-11,False


# Prepare

In [4]:
def prepare_kobis(df):
    df['year'] = df['targetDt_f'].apply(lambda x: x.year)
    df['month'] = df['targetDt_f'].apply(lambda x: x.month)
    df['audiCnt_num'] = df['audiCnt'].astype('int')
    df['audiAcc_num'] = df['audiAcc'].astype('int')
    df['showCnt_num'] = df['showCnt'].astype('int')
    df['cnt_show'] = df['audiCnt_num'] / df['showCnt_num']
    df['scrnCnt_num'] = df['scrnCnt'].astype('int')
    df['cnt_scrn'] = df['audiCnt_num'] / df['scrnCnt_num']
    
    df = df.loc[(df['targetDt_f']>=df['openDt_f'])].reset_index(drop=True)
    
    return df

box = prepare_kobis(raw)

# Functions

In [5]:
def get_movie(box, name):
    idx_movie = box['movieNm'] == name
    return box.loc[idx_movie].reset_index(drop=True)

# Get Movies

In [6]:
endgame = get_movie(box, nm_avengers4)
mr = get_movie(box, nm_myungryang)
extreme = get_movie(box, nm_extreme_job)
